In [1]:
# Geodesic upsampling finds the maximum XY displacement
# of an edge from a defined keypoint. From this maximum
# displacement, we can say the z coordinate is 0. 
# Thus, we can reconstruct the XYZ value of any edge at
# any time by calculating its XY geodesic to a keypoint
# and setting the difference between this and the maximum
# XY coordinate as the Z coordinate. Mathematically:
# z_current**2 = (x_max**2 + y_max**2) - (x_current**2 + y_current**2)
import numpy
import numba
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import face_alignment
from skimage import color
import dlib
import os, requests

In [2]:
def download_from_url(url, destination):
  r = requests.get(url, allow_redirects=True)
  open(destination, 'wb').write(r.content)
  return destination

lewis_1 = mpimg.imread( download_from_url('https://harriermain.blob.core.windows.net/data/lewis-1.png', 'lewis_1.png') )
lewis_2 = mpimg.imread( download_from_url('https://harriermain.blob.core.windows.net/data/lewis-2.png', 'lewis_2.png') )

In [3]:
plt.imshow(lewis_1)

In [4]:
plt.imshow(lewis_2)

In [5]:
def segment_faces(image_path): # Extracts unique faces from frames and places them into subfolders.
    detector = dlib.get_frontal_face_detector()
    sp = dlib.shape_predictor( download_from_url('https://harriermain.blob.core.windows.net/assets/5mark.dat', '5mark.dat') )
    facerec = dlib.face_recognition_model_v1( download_from_url('https://harriermain.blob.core.windows.net/assets/facerec.dat', 'facerec.dat') )
    frame_img = dlib.load_rgb_image(image_path)
    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(frame_img, 1)
    # Now process each face we found.
    for k, d in enumerate(dets): # For every face detected within the image:
        # Compute the 128D vector that describes the face in img identified by
        # shape. If two face vectors have a Euclidean
        # distance of less than 0.6, they are usually from the same person.
        croppedFace = frame_img[d.top():d.bottom(), d.left():d.right()]
    return croppedFace

In [6]:
lewis_1c = segment_faces('lewis_1.png')
plt.imshow(lewis_1c)

In [7]:
lewis_2c = segment_faces('lewis_2.png')
plt.imshow(lewis_2c)

In [8]:
def RGBtoNumpy(img):
    return color.rgb2gray(numpy.asanyarray(img))
  
lewis_1cn = RGBtoNumpy(lewis_1c)
lewis_2cn = RGBtoNumpy(lewis_2c)

In [9]:
plt.imshow(lewis_1cn)

In [10]:
plt.imshow(lewis_2cn)

In [11]:
@numba.njit
def run_kernel(img_numpy, threshold):
    kernel=numpy.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
    convolved_img = numpy.zeros((len(img_numpy), len(img_numpy[0])), dtype=numpy.float64)
    kernel_delta = int(numpy.floor(len(kernel)/2))
    for x in range(kernel_delta, len(img_numpy)-kernel_delta):
        for y in range(kernel_delta, len(img_numpy[0])-kernel_delta):
            element = 0
            for x_k in range(len(kernel)):
                for y_k in range(len(kernel[0])):
                    element += img_numpy[(x - kernel_delta) + x_k][(y - kernel_delta) + y_k] * kernel[x_k][y_k]
            if abs(element) >= threshold:
                convolved_img[x][y] = 1
    return convolved_img

In [12]:
lewis_grad1 = run_kernel(lewis_1cn, 0.012)
plt.imshow(lewis_grad1)

In [13]:
lewis_grad2 = run_kernel(lewis_2cn, 0.012)
plt.imshow(lewis_grad2)

In [14]:
@numba.njit
def blur_img(img_numpy):
    kernel=numpy.array([[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]])
    convolved_img = numpy.zeros((len(img_numpy), len(img_numpy[0])), dtype=numpy.float64)
    kernel_delta = int(numpy.floor(len(kernel)/2))
    for x in range(kernel_delta, len(img_numpy)-kernel_delta):
        for y in range(kernel_delta, len(img_numpy[0])-kernel_delta):
            element = 0
            for x_k in range(len(kernel)):
                for y_k in range(len(kernel[0])):
                    element += img_numpy[(x - kernel_delta) + x_k][(y - kernel_delta) + y_k] * kernel[x_k][y_k]
            convolved_img[x][y] = abs(element)
    return convolved_img

lewis_blur = blur_img(lewis_grad2)
plt.imshow(lewis_blur)

In [15]:
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._3D, flip_input=False)
def apply_3D_estimate(image): # , blob_path):
    # Using FAN, apply 3D estimate to each video frame. Form a 
    # facenet and return each frame result as JSON to CosmosDB.
    # Include pitch, roll and yaw data given by Face API.
    #response = requests.post(self.face_api_url, params=self.params, headers=self.headers, json={"url": blob_path})
    landmarks = self.fa.get_landmarks(image)
    return landmarks_list
  
print(apply_3d_estimate(lewis_1c))

--------------------------------------------------------------------------- 
 AssertionError Traceback (most recent call last)
 <command-1029793613186922> in <module> 
 ----> 1 fa = face_alignment . FaceAlignment ( face_alignment . LandmarksType . _3D , flip_input = False ) 
 2 def apply_3D_estimate ( image ) : # , blob_path): 
 3 # Using FAN, apply 3D estimate to each video frame. Form a 
 4 # facenet and return each frame result as JSON to CosmosDB. 
 5 # Include pitch, roll and yaw data given by Face API. 

 /databricks/python/lib/python3.7/site-packages/face_alignment/api.py in __init__ (self, landmarks_type, network_size, device, flip_input, face_detector, verbose) 
 64 face_detector_module = __import__('face_alignment.detection.' + face_detector,
 65 globals(), locals(), [face_detector], 0)
 ---> 66 self . face_detector = face_detector_module . FaceDetector ( device = device , verbose = verbose ) 
 67 
 68 # Initialise the face alignemnt networks 

 /databricks/python/lib/python3.7/site-packages/face_alignment/detection/sfd/sfd_detector.py in __init__ (self, device, path_to_detector, verbose) 
 43 self . face_detector = s3fd ( ) 
 44 self . face_detector . load_state_dict ( torch . load ( path_to_detector ) ) 
 ---> 45 self . face_detector . to ( device ) 
 46 self . face_detector . eval ( ) 
 47 

 /databricks/python/lib/python3.7/site-packages/torch/nn/modules/module.py in to (self, *args, **kwargs) 
 441 return t . to ( device , dtype if t . is_floating_point ( ) else None , non_blocking ) 
 442 
 --> 443 return self . _apply ( convert ) 
 444 
 445 def register_backward_hook ( self , hook ) : 

 /databricks/python/lib/python3.7/site-packages/torch/nn/modules/module.py in _apply (self, fn) 
 201 def _apply ( self , fn ) : 
 202 for module in self . children ( ) : 
 --> 203 module . _apply ( fn ) 
 204 
 205 def compute_should_use_set_data ( tensor , tensor_applied ) : 

 /databricks/python/lib/python3.7/site-packages/torch/nn/modules/module.py in _apply (self, fn) 
 223 # `with torch.no_grad():` 
 224 with torch . no_grad ( ) : 
 --> 225 param_applied = fn ( param ) 
 226 should_use_set_data = compute_should_use_set_data ( param , param_applied ) 
 227 if should_use_set_data : 

 /databricks/python/lib/python3.7/site-packages/torch/nn/modules/module.py in convert (t) 
 439 if convert_to_format is not None and t . dim ( ) == 4 : 
 440 return t . to ( device , dtype if t . is_floating_point ( ) else None , non_blocking , memory_format = convert_to_format ) 
 --> 441 return t . to ( device , dtype if t . is_floating_point ( ) else None , non_blocking ) 
 442 
 443 return self . _apply ( convert ) 

 /databricks/python/lib/python3.7/site-packages/torch/cuda/__init__.py in _lazy_init () 
 147 raise RuntimeError(
 148 "Cannot re-initialize CUDA in forked subprocess. " + msg)
 --> 149 _check_driver ( ) 
 150 if _cudart is None : 
 151 raise AssertionError(

 /databricks/python/lib/python3.7/site-packages/torch/cuda/__init__.py in _check_driver () 
 45 def _check_driver ( ) : 
 46 if not hasattr ( torch . _C , '_cuda_isDriverSufficient' ) : 
 ---> 47 raise AssertionError ( "Torch not compiled with CUDA enabled" ) 
 48 if not torch . _C . _cuda_isDriverSufficient ( ) : 
 49 if torch . _C . _cuda_getDriverVersion ( ) == 0 : 

 AssertionError : Torch not compiled with CUDA enabled

In [16]:
nosetip_array = nosetip_array # 1D list of nosetips for each frame; nosetip_array[frame_number]
super_edge_list = edge_list # 2D list of frame edges; super_edge_list[frame_number][edge_number]
matched_edges = [] # 2D list of edges that are found throughout multiple frames; matched_edges[edge_id][frame_number]

def normalise_edge(edge_list):
    # Rescale every edge to length 1. This allows 
    # for uniform comparison.
    edge_geodesic = 0
    for count in range(1, len(edge_list)):
        edge_geodesic += (edge_list[count-1])**2 + (edge_list[count])**2
    edge_geodesic **= 0.5
    for x in range(len(edge_list)):
        edge_list[x] /= edge_geodesic
    return edge_list

In [17]:
def track_edges():
# Blur edges and compile onto single frame.
# Calculate overlap of edges in adjacent frames,
# as a normalised square mean of overlap pixels.
# If blurs overlap in adjacent frames is sufficient,
# (>0.9), then edges are the same and can be
# geodesically upsampled.
for frame in range(1, len(self.super_edge_list)):
    edge_list1 = self.super_edge_list[frame-1]
    edge_list2 = self.super_edge_list[frame]
    edge_list1 = self.normalise_edge(edge_list1)
    edge_list2 = self.normalise_edge(edge_list2)
    for edge_number, edge1 in enumerate(edge_list1):
        similarity_max = 0
        matched_edge = None
        for edge2 in edge_list2:
            counter = 0
            square_sum = 0
            # num_vertices is the number of vertices that both
            # edges can be compared by. Additional vertices show 
            # the edges are of increasingly different shape: for 
            # every unshared vertex between edges, the similarity
            # measure is reduced.
            if len(edge1) < len(edge2):
                num_vertices = len(edge1)
                num_vertices_delta = len(edge2) - num_vertices
            else:
                num_vertices = len(edge2)
                num_vertices_delta = len(edge1) - num_vertices
            for y in range(num_vertices):
                # Compute Euclidian distance between points.
                # Find similarity through inverse square law of relation.
                mid_x = edge1[y][0] - edge2[y][0]
                mid_x **= 2
                mid_y = edge1[y][1] - edge2[y][1]
                mid_y **= 2
                square_sum += 1/(mix_x + mid_y + 1)
                counter += 1
            for z in range(num_vertices_delta):
                # Similarity is taxed for every vertex that 
                # cannot be matched.
                counter += 1
            similarity = square_sum / counter
            if similarity > similarity_max and similarity > 0.9:
                similarity_max = similarity
                matched_edge = edge2
        if matched_edge == None:
            # 'frame' is a frame reference used later to find the appropriate nosetip.
            self.matched_edges[len(self.matched_edges)+1].append((edge1, frame))
        else:
            self.matched_edges[self.matched_edges.index(edge1)].append((matched_edge, frame))
        self.super_edge_list[frame][edge_number] = [self.super_edge_list[frame][edge_number], self.matched_edges.index(edge1)]

In [18]:
def compute_max_geodesics(self):
        for edge_list in self.matched_edges: # For every edge identified throughout multiple frames:
            max_geodesic = 0 # Let's start here.
            for edge in edge_list: # For every variation of the identified edge:
                edge_array = edge[0] # Remember the edge is saved as [(x,y), frame_id]
                nosetip = self.nosetip_array[edge[1]] # Find the nosetip according to frame id.
                geodesic = (edge_array[0] - nosetip[0])**2 
                geodesic += (edge_array[1] - nosetip[1])**2
                if geodesic > max_geodesic:
                    max_geodesic = geodesic
            max_geodesic **= 0.5
            edge_list.append(max_geodesic)
            max_geodesic_list.append(max_geodesic) # This is the largest XY distance of this edge to nosetip.
        return max_geodesic_list

In [19]:
def reconstruct_3D_edges(self, index, fa_3D_depth):
    # Search through matched_edges and match max_geodesics
    # to values in frame via frame index. This is for creating a dataset only:
    # don't use this for 3D reconstruction of previously unprocessed frames.
    edge_3D = [[]]
    for iterator, edge in enumerate(self.super_edge_list[index]):
        edge_data = edge[0]
        edge_reference = edge[1]
        max_geodesic = self.matched_edges[edge_reference][-1]
        nosetip = self.nosetip_array[index]
        for edge_point in edge_data:
            depth = (nosetip[0] - edge_point[0])**2
            depth += (nosetip[1] - edge_point[1])**2
            depth **= 0.5
            depth += fa_3D_depth
            edge_3D[iterator].append([edge_point[0], edge_point[1], depth])
    return edge_3D